## HotChili Analytics trading notebook template
#### Configure by setting ALGO_NAME in cell below.
#### Run various options (backtest, ingest, live) by uncommenting one cell 

In [1]:
%matplotlib inline
%load_ext zipline

# %reload_ext zipline # Uncomment and use this when already loaded zipline extension for magic cell usage.

/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


In [2]:
import pandas as pd

# Options you can uncomment and set:

# pd.set_option("max_colwidth", 300)
# pd.set_option("display.max_rows", 300)
# pd.set_option("display.max_columns", 50)
# pd.set_option('precision', 2)
# pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
import os

hca_root_path = os.environ['HCA_ROOT']
print(f"hca_root_path = {hca_root_path}")

hca_root_path = /home/hca-ws2004/hca


# Construct algorithm strategy path names

Assumptions:

- the strategy is in a directory with the same name as the strategy in `ALGO_NAME` below
- the strategy is located in the hca-resources directory, which is located relative to `hca_root_path`, found above

In [4]:
ALGO_NAME = "HCA_Fixed_Ratio_Allocations" # <--- Supply name here

Other variables are derived from `ALGO_NAME`:

In [5]:
HCA_RESOURCES_PATH = hca_root_path + "/hca-resources/" 
ALGO_PATH          = HCA_RESOURCES_PATH + ALGO_NAME + "/" 
ALGO_BT            = ALGO_PATH + ALGO_NAME + ".py"
ALGO_BT_OUT        = ALGO_PATH + ALGO_NAME + ".pkl"
ALGO_LIVE          = ALGO_PATH + ALGO_NAME + "_Live" + ".py"

print(f"""
ALGO_NAME          = {ALGO_NAME}
HCA_RESOURCES_PATH = {HCA_RESOURCES_PATH}
ALGO_PATH          = {ALGO_PATH}
ALGO_BT            = {ALGO_BT}
ALGO_BT_OUT        = {ALGO_BT_OUT}
ALGO_LIVE          = {ALGO_LIVE}

Contents of algo directory:
""")

!ls $ALGO_PATH


ALGO_NAME          = HCA_Fixed_Ratio_Allocations
HCA_RESOURCES_PATH = /home/hca-ws2004/hca/hca-resources/
ALGO_PATH          = /home/hca-ws2004/hca/hca-resources/HCA_Fixed_Ratio_Allocations/
ALGO_BT            = /home/hca-ws2004/hca/hca-resources/HCA_Fixed_Ratio_Allocations/HCA_Fixed_Ratio_Allocations.py
ALGO_BT_OUT        = /home/hca-ws2004/hca/hca-resources/HCA_Fixed_Ratio_Allocations/HCA_Fixed_Ratio_Allocations.pkl
ALGO_LIVE          = /home/hca-ws2004/hca/hca-resources/HCA_Fixed_Ratio_Allocations/HCA_Fixed_Ratio_Allocations_Live.py

Contents of algo directory:

HCA_Fixed_Ratio_Allocations.ipynb
HCA_Fixed_Ratio_Allocations_Live.py
HCA_Fixed_Ratio_Allocations.pkl
HCA_Fixed_Ratio_Allocations.py
HCA_Fixed_Ratio_Allocations_tearsheet.ipynb


## Zipline backtest: 

- Method: Jupyter magic cell (%%) 
- Execution of zipline code in cell containing command line command
- Uncomment first line and hit (shift-enter) inside the cell to run simulation backtest

In [6]:
#%%zipline --start=2018-1-1 --end=2020-10-02 -b sharadar-funds -o $ALGO_BT_OUT

# Source: adapted from various algos on quantopian
# HCA Conversion Date: 08-13-2020
# Conversion Author: Anthony garner

# Simple rebalanced portfolio


import matplotlib.pyplot as plt

from zipline.api import (symbol, symbols, date_rules, time_rules, order_target_percent, 
                         record, schedule_function, get_datetime)
from trading_calendars import get_calendar


def initialize(context):
    schedule_function(func=trade,date_rule=date_rules.every_day(),time_rule=time_rules.market_open(),half_days=True)
    context.asserts = symbols('SPY','SHY','TLT','GLD')
    context.asserts_position = [0.25, 0.25,0.25,0.25]
    context.rebalance_inteval = 'Q'#'Q', #'D', #'M' #'Q' #'Y'
    context.rebalance_date = 0
    context.fired = False


def handle_data(context, data):
    record(SPY=data[symbol('SPY')].price)


def is_new_day(context, now):
    return ( (now.year > context.rebalance_date.year) or (now.month > context.rebalance_date.month) or((now.month == context.rebalance_date.month) and (now.day > context.rebalance_date.day)))             


def is_new_month(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month > context.rebalance_date.month)))


def is_new_quarter(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month == context.rebalance_date.month + 3)))


def is_new_year(context, now):
    return (now.year > context.rebalance_date.year)


def need_rebalance(context, now):
    return ((context.rebalance_inteval == 'Y' and is_new_year(context, now))or 
           (context.rebalance_inteval == 'Q' and is_new_quarter(context, now)) or 
           (context.rebalance_inteval == 'M' and is_new_month(context, now)) or 
           (context.rebalance_inteval == 'D' and is_new_day(context, now)))


def init_portfolio(context, data):
    for i in range(0, len(context.asserts)):
        if data.can_trade(context.asserts[i]):
            #log.debug("rebalance " + context.asserts[i].symbol + " to:" + str(context.asserts_position[i]*100) + "%")
            order_target_percent(context.asserts[i], context.asserts_position[i])


def rebalance(context, data):
    for i in range(0, len(context.asserts)):
        if data.can_trade(context.asserts[i]):
            #log.debug("rebalance " + context.asserts[i].symbol + " to:" + str(context.asserts_position[i]*100) + "%")
            order_target_percent(context.asserts[i], context.asserts_position[i])
        

# Will be called on every trade event for the securities you specify. 
def trade(context, data):
    if not context.fired:
        context.rebalance_date = get_datetime()
        #log.info("build portfolio at " + str(context.rebalance_date))
        init_portfolio(context, data)
        context.fired = True
        now = get_datetime()
    else:
        now = get_datetime()
        if (need_rebalance(context, now)):
            #log.info("new rebalance arrivied:" + str(now))
            rebalance(context, data)
            context.rebalance_date = now


def analyze(context, perf):
    ax1 = plt.subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax2 = plt.subplot(212, sharex=ax1)
    perf.SPY.plot(ax=ax2)
    plt.gcf().set_size_inches(18, 8)
    plt.show()

## Display your current bundles

In [7]:
#!zipline bundles # Finds all bundles

## Ingest Sharadar funds assets for today

In [8]:
# Ingest Sharadar funds assets for today, if needed.

# Only need to ingest Funds for this algo, and this takes less processing time and system memory than ingesting
# all of Sharadar Equities plus Funds bundle (sharadar-eqfd)

#!zipline ingest -b sharadar-funds

# Yahoo Finance Free Data
# Trend_Following
# !export YAHOO_SYM_LST=SPY,ZSL,KOLD,GLD,SHY;zipline ingest -b yahoo_direct
# Fixed_Ratio_Allocations
!export YAHOO_SYM_LST=QQQ,XLP,IEF,TLT;zipline ingest -b yahoo_direct


/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
extension: hca_root_path = /home/hca-ws2004/hca
extension:TODAY_STR = 2021-04-10
extension:TWO_YR_AGO_STR = 2019-04-10
extension:  start_date=2019-04-10 end_date = 2021-04-10
[2021-04-11 03:26:28.611003] INFO: zipline.data.bundles.ingester: symbols are: ('TLT', 'IEF', 'XLP', 'QQQ')
[2021-04-11 03:26:28.611982] INFO: zipline.data.bundles.ingester: writing data...
Yahoo

## Zipline backtest, alternative method

- Method: command line
- Execution of zipline code, located in a file, using below command line execution with magic (`!`) invocation
- This line can also be run in a terminal by copying everything past the `!` and pasting (shift-insert) it into the target terminal

In [13]:
!zipline run -f $ALGO_BT  --start=2018-1-1 --end=2021-04-10 --capital-base 10000 -b sharadar-funds -o $ALGO_BT_OUT

/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
extension: hca_root_path = /home/hca-ws2004/hca
extension:TODAY_STR = 2021-04-10
extension:TWO_YR_AGO_STR = 2019-04-10
extension:  start_date=2019-04-10 end_date = 2021-04-10
[2021-04-11 03:32:26.735299] WARNING: Loader: Refusing to download new benchmark data because a download succeeded at 2021-04-11 03:17:05.084462+00:00.
[2021-04-11 03:32:26.764731] WARNING: Loade

## Run Zipline live on IB-TWS via command line

- Method: command line
- Execution of zipline code using below command line execution using magic (`!`) invocation

**Notes:** 
- IB-TWS or IB-Gateway must be running, with `IB_ACCT` and `IB_URI` port being correct to live trade
- Change `I_WANT_TO_RUN_THIS_CODE` to `True` below to run zipline live on IB-TWS/IB-Gateway

In [10]:
TODAY = pd.datetime.today().strftime("%Y-%m-%d")
print("TODAY = {}".format(TODAY))

TODAY = 2021-04-10


In [11]:
ALGO_STATE = ALGO_PATH + "strategy.state" 
ALGO_RTB   = ALGO_PATH + "realtime-bars/"

# Edit the following URI to match your IB account and port info.
BROKER = "ib"
IB_ACCT = "DU1568488"
IB_URI = "127.0.0.1:7497:1301"

#  PRICES_BUNDLE = "hca-symbol" #Used for Zipline Symbols to Live Broker Mapping(e.g. IB)
PRICES_BUNDLE = "yahoo_direct" #Used for free data bundle for zipline.
DATA_FREQ = "daily"

# Change following to 'True' and run cell (control-enter) to execute live run.
I_WANT_TO_RUN_THIS_CODE = False
# I_WANT_TO_RUN_THIS_CODE = True

if I_WANT_TO_RUN_THIS_CODE:
    
    !zipline run \
        -s $TODAY \
        -f $ALGO_LIVE \
        --bundle $PRICES_BUNDLE \
        --broker $BROKER \
        --broker-uri $IB_URI \
        --broker-acct $IB_ACCT \
        --data-frequency $DATA_FREQ \
        --state-file $ALGO_STATE \
        --realtime-bar-target $ALGO_RTB 

In [12]:
# This bundle is write-protected, and not to be updated. It will produce a write error.
# Used only as a zipline-to-live-broker symbol mapping 
#  and for older backtests dating from the bundle production date(2020-12-06T23;31;54.306657)
#! zipline ingest -b hca-symbol